In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import col

import sys
import os
os.environ["PYSPARK_PYTHON"] = sys.executable

# 1. Crear sesión Spark
spark = SparkSession.builder.appName("AnalisisCarreras").getOrCreate()

# 2. Cargar datos en DataFrame
df = spark.read.json("carreras.json")
df.show()

+---------------+--------------------+--------------------+----+---------+----------------+
|_corrupt_record|                area|             carrera|  id|inscritos|     universidad|
+---------------+--------------------+--------------------+----+---------+----------------+
|              [|                NULL|                NULL|NULL|     NULL|            NULL|
|           NULL|Económicas y Admi...|Ingeniería Comercial|   1|     3200|     U. de Chile|
|           NULL|   Ciencias Sociales|             Derecho|   2|     2900|             PUC|
|           NULL|               Salud|            Medicina|   3|     2500|U. de Concepción|
|           NULL|   Ciencias Sociales|          Psicología|   4|     1800|  U. de Santiago|
|           NULL|Ingeniería y Tecn...|    Ingeniería Civil|   5|     3100|           UTFSM|
|              ]|                NULL|                NULL|NULL|     NULL|            NULL|
+---------------+--------------------+--------------------+----+---------+------

In [2]:
# 3. Imprimir esquema
df.printSchema()

# Fix: Changed the temporary view name to a single-part name.
df.createOrReplaceTempView("carreras")

root
 |-- _corrupt_record: string (nullable = true)
 |-- area: string (nullable = true)
 |-- carrera: string (nullable = true)
 |-- id: long (nullable = true)
 |-- inscritos: long (nullable = true)
 |-- universidad: string (nullable = true)



In [3]:
# 4a. Carreras con más de 2500 inscritos
print("Carreras con más de 2500 inscritos:")
spark.sql("SELECT * FROM carreras WHERE inscritos > 2500").show()

# 4b. Cantidad de carreras por área
print("Cantidad de carreras por área:")
spark.sql("SELECT area, COUNT(*) as cantidad FROM carreras GROUP BY area").show()

# 4c. Universidades con más de una carrera
print("Universidades con más de una carrera:")
spark.sql("SELECT universidad, COUNT(*) as num_carreras FROM carreras GROUP BY universidad HAVING num_carreras > 1").show()

Carreras con más de 2500 inscritos:
+---------------+--------------------+--------------------+---+---------+-----------+
|_corrupt_record|                area|             carrera| id|inscritos|universidad|
+---------------+--------------------+--------------------+---+---------+-----------+
|           NULL|Económicas y Admi...|Ingeniería Comercial|  1|     3200|U. de Chile|
|           NULL|   Ciencias Sociales|             Derecho|  2|     2900|        PUC|
|           NULL|Ingeniería y Tecn...|    Ingeniería Civil|  5|     3100|      UTFSM|
+---------------+--------------------+--------------------+---+---------+-----------+

Cantidad de carreras por área:
+--------------------+--------+
|                area|cantidad|
+--------------------+--------+
|               Salud|       1|
|Ingeniería y Tecn...|       1|
|Económicas y Admi...|       1|
|   Ciencias Sociales|       2|
|                NULL|       2|
+--------------------+--------+

Universidades con más de una carrera:
+--

In [4]:
# 5. UDF para clasificar demanda
@udf(StringType())
def clasificar_demanda(inscritos):
    if inscritos is None:
        return "Desconocida"  # Handle None values
    if inscritos > 3000:
        return "Alta demanda"
    elif inscritos >= 2000:
        return "Media demanda"
    else:
        return "Baja demanda"

# Filter out rows where 'inscritos' is None before applying the UDF
df_filtered = df.filter(col("inscritos").isNotNull())

df2 = df_filtered.withColumn("demanda", clasificar_demanda(df_filtered.inscritos))
df2.show()

# 6. Guardar en Parquet
df2.write.mode("overwrite").parquet("carreras_procesadas.parquet")

spark.stop()

+---------------+--------------------+--------------------+---+---------+----------------+-------------+
|_corrupt_record|                area|             carrera| id|inscritos|     universidad|      demanda|
+---------------+--------------------+--------------------+---+---------+----------------+-------------+
|           NULL|Económicas y Admi...|Ingeniería Comercial|  1|     3200|     U. de Chile| Alta demanda|
|           NULL|   Ciencias Sociales|             Derecho|  2|     2900|             PUC|Media demanda|
|           NULL|               Salud|            Medicina|  3|     2500|U. de Concepción|Media demanda|
|           NULL|   Ciencias Sociales|          Psicología|  4|     1800|  U. de Santiago| Baja demanda|
|           NULL|Ingeniería y Tecn...|    Ingeniería Civil|  5|     3100|           UTFSM| Alta demanda|
+---------------+--------------------+--------------------+---+---------+----------------+-------------+

